In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide_310/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide_310/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-pre

In [2]:
protein = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
peptide = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
supplemental = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [3]:
#protein & peptide 에 labelencoder 적용
protein.isnull().sum()

visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64

In [4]:
peptide.isnull().sum()

visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64

In [5]:
# labelencoder로 protein의 UniProt, peptide의 Peptide column 라벨링
# 이때, protein과 peptide data의 UniProt을 같은 기준으로 라벨링함
# from sklearn.preprocessing import LabelEncoder
# l = LabelEncoder()
# n = LabelEncoder()
# protein["UniProt"] = l.fit_transform(protein["UniProt"])
# peptide["UniProt"] = l.transform(peptide["UniProt"])
# peptide["Peptide"] = n.fit_transform(peptide["Peptide"])

In [6]:
protein

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [7]:
peptide

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [8]:
#clinical & supplemental category변환 및 결측치 처리
clinical.isnull().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [9]:
supplemental.isnull().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                 213
updrs_2                                 214
updrs_3                                   5
updrs_4                                 928
upd23b_clinical_state_on_medication    1101
dtype: int64

In [10]:
# clinical data의 On->1, Off->0 으로 치환, clinical state의 결측치를 0으로 처리
clinical['upd23b_clinical_state_on_medication'].replace('On', 1, inplace=True)
clinical['upd23b_clinical_state_on_medication'].replace('Off', 0 , inplace=True)
clinical = clinical.fillna({'upd23b_clinical_state_on_medication':0})

In [11]:
clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,0.0
1,55_3,55,3,10.0,7.0,25.0,NaN,0.0
2,55_6,55,6,8.0,10.0,34.0,NaN,0.0
3,55_9,55,9,8.0,9.0,30.0,0.0,1.0
4,55_12,55,12,10.0,10.0,41.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0,0.0
2611,65043_54,65043,54,4.0,8.0,11.0,1.0,0.0
2612,65043_60,65043,60,6.0,6.0,16.0,1.0,0.0
2613,65043_72,65043,72,3.0,9.0,14.0,1.0,0.0


In [12]:
# supplemental data의 On->1, Off->0 으로 치환, clinical state의 결측치를 0으로 처리
supplemental['upd23b_clinical_state_on_medication'].replace('On', 1, inplace=True)
supplemental['upd23b_clinical_state_on_medication'].replace('Off', 0 , inplace=True)
supplemental = supplemental.fillna({'upd23b_clinical_state_on_medication':0})

In [13]:
# clinical & supplemental data에 보간법 적용 -> updrs 결측치 처리
# clinical['updrs_1'] = clinical['updrs_1'].interpolate(method='linear', axis=0)
# clinical['updrs_2'] = clinical['updrs_2'].interpolate(method='linear', axis=0)
# clinical['updrs_3'] = clinical['updrs_3'].interpolate(method='linear', axis=0)
# clinical['updrs_4'] = clinical['updrs_4'].interpolate(method='linear', axis=0)
# -----------------------------------------------------------------------------
supplemental['updrs_1'] = supplemental['updrs_1'].interpolate(method='linear', axis=0)
supplemental['updrs_2'] = supplemental['updrs_1'].interpolate(method='linear', axis=0)
supplemental['updrs_3'] = supplemental['updrs_1'].interpolate(method='linear', axis=0)
supplemental['updrs_4'] = supplemental['updrs_1'].interpolate(method='linear', axis=0)

In [14]:
clinical.isnull().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication       0
dtype: int64

In [15]:
clinical = clinical.fillna({'updrs_4':0})

In [16]:
clinical.isnull().sum()

visit_id                                0
patient_id                              0
visit_month                             0
updrs_1                                 1
updrs_2                                 2
updrs_3                                25
updrs_4                                 0
upd23b_clinical_state_on_medication     0
dtype: int64

In [17]:
supplemental.isnull().sum()

visit_id                               0
patient_id                             0
visit_month                            0
updrs_1                                0
updrs_2                                0
updrs_3                                0
updrs_4                                0
upd23b_clinical_state_on_medication    0
dtype: int64

In [18]:
#Merge 진행
# pp = protein.merge(peptide[['visit_id', 'UniProt', 'Peptide','PeptideAbundance']],
                    # on = ['visit_id','UniProt'], how = 'left')
# cs = pd.concat([clinical, supplemental]).drop_duplicates()
# df = pp.merge(cs[['visit_id', 'updrs_1', 'updrs_2','updrs_3','updrs_4','upd23b_clinical_state_on_medication']],
              # on = ['visit_id'], how = 'left')

In [19]:
# df = df.dropna()
# df.isnull().sum()

protein+peptide : train_X, train_clinical.csv: train_y
pivot을 이용해 UniProt, Peptide를 열로 만들기

In [20]:
#NPX scaling, train data만들기
def normalize(column):
  min_val = np.min(column)
  max_val = np.max(column)
  normalized = (column-min_val)/(max_val-min_val)
  return normalized

# protein['NPX'] = normalize(protein['NPX'])
# peptide['PeptideAbundance'] = normalize(peptide['PeptideAbundance'])


In [21]:
peptide

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.30
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.00
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.00
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.70
...,...,...,...,...,...,...
981829,58648_108,108,58648,Q9UHG2,ILAGSADSEGVAAPR,202820.00
981830,58648_108,108,58648,Q9UKV8,SGNIPAGTTVDTK,105830.00
981831,58648_108,108,58648,Q9Y646,LALLVDTVGPR,21257.60
981832,58648_108,108,58648,Q9Y6R7,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26


In [22]:
def prepro(proteins, peptides):
    pro_group = proteins.groupby(['visit_id','UniProt'])['NPX'].mean().reset_index()
    pep_group = peptides.groupby(['visit_id','Peptide'])['PeptideAbundance'].mean().reset_index()
    
    #pivot
    pro = pro_group.pivot(index='visit_id',columns='UniProt',values='NPX').rename_axis(columns=None).reset_index()
    pep = pep_group.pivot(index='visit_id',columns='Peptide',values = 'PeptideAbundance').rename_axis(columns=None).reset_index()
    print(pro.info())
    print(pep.info())
    pro_pep_df = pro.merge(pep, on = ['visit_id'], how = 'left')
    
    return pro_pep_df

In [23]:
df = prepro(protein, peptide)
# df = protein

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Columns: 228 entries, visit_id to Q9Y6R7
dtypes: float64(227), object(1)
memory usage: 1.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Columns: 969 entries, visit_id to YYWGGQYTWDMAK
dtypes: float64(968), object(1)
memory usage: 8.2+ MB
None


In [24]:
df

,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
1109,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN
1110,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,185428.0,5554.53,NaN,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60
1111,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,137611.0,6310.09,NaN,28008.8,231359.0,63265.8,64601.8,632782.0,51123.7,20700.30


In [25]:
# def convertUni(df):
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^O0', value = 0, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^O1', value = 1, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^O4', value = 2, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^O6', value = 3, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^O7', value = 4, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^O9', value = 5, regex = True)

# #P로 시작하는 코드가 많아서 P의 경우 세번째자리까지 해서 카테고리를 나눔
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P00', value = 6, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P01', value = 7, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P02', value = 8, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P04', value = 9, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P05', value = 10, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P06', value = 11, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P07', value = 12, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P08', value = 13, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P09', value = 14, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P10', value = 15, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P11', value = 16, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P12', value = 17, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P13', value = 18, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P14', value = 19, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P16', value = 20, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P17', value = 21, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P18', value = 22, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P19', value = 23, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P20', value = 24, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P23', value = 25, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P24', value =26, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P25', value = 27, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P27', value = 28, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P30', value = 29, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P31', value = 30, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P32', value = 31, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P35', value = 32, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P36', value = 33, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P39', value = 34, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P40', value = 35, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P41', value = 36, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P43', value = 37, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P49', value = 38, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P51', value = 39, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P54', value = 40, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P55', value = 41, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P60', value = 42, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P61', value = 43, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P80', value = 44, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^P98', value = 45, regex = True)

#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q0', value = 46, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q1', value = 47, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q5', value = 48, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q6', value = 49, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q7', value = 50, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q8', value = 51, regex = True)
#     df['UniProt'] = df['UniProt'].replace(to_replace=r'^Q9', value = 52, regex = True)
    
#     return df

In [26]:
# df = convertUni(df)

In [27]:
df

,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
1109,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN
1110,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,185428.0,5554.53,NaN,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60
1111,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,137611.0,6310.09,NaN,28008.8,231359.0,63265.8,64601.8,632782.0,51123.7,20700.30


In [28]:
# pro_group = df.groupby(['visit_id','patient_id','visit_month','UniProt'])['NPX'].mean().reset_index()
# pro = pro_group.pivot(index='visit_id',columns='UniProt',values='NPX').rename_axis(columns=None).reset_index()

In [29]:
#train- XGBoost 이용
# SMAPE 계산 함수 정의
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    return np.mean(diff) * 100

In [30]:
# df = pro.merge(clinical[['visit_id','patient_id','visit_month', 'updrs_1','updrs_2','updrs_3','updrs_4']])
df = df.merge(clinical[['visit_id','patient_id','visit_month', 'updrs_1','updrs_2','updrs_3','updrs_4']])

In [31]:
df.fillna(0, inplace=True)

In [32]:
df.columns = df.columns.astype(str)

In [33]:
df.isnull().sum()

visit_id       0
O00391         0
O00533         0
O00584         0
O14498         0
              ..
visit_month    0
updrs_1        0
updrs_2        0
updrs_3        0
updrs_4        0
Length: 1202, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

# 데이터 분할
X = df.drop(['visit_id','updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'], axis=1)
y = df[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = {}

params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': 'gbtree',
        'tree_method': 'hist',
        'grow_policy': 'lossguide',
    }

#XGRegressor와 RandomForestRegressor 
xg_reg1 = XGBRegressor(**params)
xg_reg3 = XGBRegressor(**params)
rf_reg2 = RandomForestRegressor()
rf_reg4 = RandomForestRegressor()

# 모델 파라미터 수정
# xg_reg.max_depth = 14
# xg_reg.eta = 0.1
# xg_reg.subsample = 0.94
# xg_reg.colsample_bytree = 0.94
# xg_reg.min_child_weight = 5
# xg_reg.reg_alpha = 0.001
# xg_reg.reg_lambda = 0.001
# xg_reg.gamma = 0.239

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [35]:
ensemble = VotingRegressor([
    ('xg1', xg_reg1), 
    ('rf2', rf_reg2),
    ('xg3', xg_reg3),
    ('rf4', rf_reg4)
])

# updrs 파트 1과 3을 XGBRegressor로 학습
a = xg_reg1.fit(X_train, y_train.iloc[:, 0])
model["updrs_1"] = a
b = xg_reg3.fit(X_train, y_train.iloc[:, 2])
model["updrs_2"] = b

# updrs 2와 4를 RandomForestRegressor로 학습
c = rf_reg2.fit(X_train, y_train.iloc[:, 1])
model["updrs_3"] = c
d = rf_reg4.fit(X_train, y_train.iloc[:, 3])
model["updrs_4"] = d

# 테스트 데이터에 대한 예측 수행
y_pred1 = xg_reg1.predict(X_test) # 예측값 1을 XGBRegressor로 예측
y_pred2 = rf_reg2.predict(X_test)  # 예측값 2를 RandomForestRegressor로 예측
y_pred3 = xg_reg3.predict(X_test) # 예측값 3을 XGBRegressor로 예측
y_pred4 = rf_reg4.predict(X_test)  # 예측값 4를 RandomForestRegressor로 예측

# 네 개의 예측 결과를 하나의 배열로 결합
y_pred = np.column_stack((y_pred1, y_pred2, y_pred3, y_pred4))

# SMAPE 계산
smape_score = smape(y_test, y_pred)
print("SMAPE score:", smape_score)

SMAPE score: updrs_1     71.145910
updrs_2    100.029388
updrs_3     94.554750
updrs_4    179.322008
dtype: float64


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [36]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854 entries, 910 to 860
Columns: 1197 entries, O00391 to visit_month
dtypes: float64(1195), int64(2)
memory usage: 7.8 MB


In [37]:
features = [i for i in X_train.columns]
len(features)

1197

In [38]:
#LSTM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

# # 데이터 준비
# X = df[['visit_month']].values
# y_updrs_1 = df['updrs_1'].values
# y_updrs_2 = df['updrs_2'].values
# y_updrs_3 = df['updrs_3'].values
# y_updrs_4 = df['updrs_4'].values

# # 데이터 전처리 (정규화)
# X = (X - np.mean(X)) / np.std(X)

# # 시퀀스 길이 설정
# sequence_length = 6

# # LSTM 입력 데이터 형태로 변환
# X_lstm = []
# y_lstm = []
# for i in range(len(X) - sequence_length):
#     X_lstm.append(X[i:i+sequence_length])
#     y_lstm.append(y[i+sequence_length])
# X_lstm = np.array(X_lstm)
# y_lstm = np.array(y_lstm)

# # 훈련 데이터와 테스트 데이터 분리
# train_size = int(len(X_lstm) * 0.8)
# X_train, X_test = X_lstm[:train_size], X_lstm[train_size:]
# y_train, y_test = y_lstm[:train_size], y_lstm[train_size:]

# # LSTM 모델 구성
# lstm = Sequential()
# lstm.add(LSTM(64, input_shape=(sequence_length, 1)))
# lstm.add(Dense(4))  # 출력 뉴런 개수는 다중 타겟의 개수에 맞춰 설정
# lstm.compile(loss='mse', optimizer='adam')

# # 모델 훈련
# mod_lstm = lstm.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# # 테스트 데이터로 예측 수행
# y_pred = lstm.predict(X_test)

# smape_score = smape(y_test, y_pred)
# print("SMAPE score:", smape_score)

In [39]:
import sys
sys.path.append('/kaggle/input/amp-pd/')

In [40]:
import amp_pd_peptide_310
env = amp_pd_peptide_310.make_env()
iter_test = env.iter_test()

df: test_protein+test_peptide
test: test.csv
model: 저장한 모델

주어진 test.csv의 예측은 앙상블 모델로, plus month는 lstm기반 모델로 진행
전자의 경우, 모델 인풋 사이즈 맞아야해서 FEATURE변수에 모델에서 사용한 열? 특성 모두 저장해놓고 그 feature에 해당하는 애만 testdf에서 뽑아서 모델 돌리기 

In [41]:
def make_prediction(test, model): 
    target = ['updrs_1', 'updrs_2','updrs_3','updrs_4']

    for u in target:
        test['result_' + str(u)] = 0
        
        existing_features = [col for col in features if col in test.columns]
        X = test[existing_features]
        for feature in features:
            if feature not in existing_features:
                X[feature] = 0
        # column_order = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', 'patient_id', 'visit_month']
        X = X.reindex(columns=features)
        test['result_' + str(u)] = model[u].predict(X)

        
    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:

            temp = test[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp [['prediction_id', 'rating']]

            result = result.append(temp)            
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

In [42]:
# def make_time_prediction():
    #LSTM으로 plus month 값 예측

In [43]:
#test 데이터
# test = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv")
# test_peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv")
# test_proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv")

In [44]:
# test_proteins = convertUni(test_proteins)
# test_proteins['UniProt']

In [45]:
# test_df = test_proteins

In [46]:
# test_df.fillna(0, inplace=True)

In [47]:
# test_group = test_df.groupby(['visit_id','patient_id','visit_month','UniProt'])['NPX'].mean().reset_index()
# tt = test_group.pivot(index='visit_id',columns='UniProt',values='NPX').rename_axis(columns=None).reset_index()

In [48]:
# test_df = test.merge(tt, how='left', on='visit_id')

In [49]:
# test_df.columns = test_df.columns.astype(str)

In [50]:
# test_df.fillna(0, inplace=True)
# test_df

동일한 patient_id에 대하여, visit_month를 통해 단백질 값을 예측..?
일단은 0으로 채우기..

In [51]:
# result = make_prediction(test_df, model)
# env.predict(result)

# print(result)

In [52]:
def sort(df):
    df['Chunks'] = df.iloc[:, 0].str.split('_')
    df['Sorting_Criteria'] = df['Chunks'].apply(lambda x: [int(i) if i.isdigit() else i for i in x])
    sorted_df = df.sort_values(by='Sorting_Criteria')
    sorted_df = sorted_df.drop(['Chunks','Sorting_Criteria'], axis=1)
    sorted_df['rating'] = np.clip(sorted_df['rating'], 0, 100)
    
    return sorted_df

In [53]:
def prepare(test, test_peptides, test_proteins):
    pro_group = test_proteins.groupby(['visit_id','UniProt'])['NPX'].mean().reset_index()
    pep_group = test_peptides.groupby(['visit_id','Peptide'])['PeptideAbundance'].mean().reset_index()
    
    #pivot
    pro = pro_group.pivot(index='visit_id',columns='UniProt',values='NPX').rename_axis(columns=None).reset_index()
    pep = pep_group.pivot(index='visit_id',columns='Peptide',values = 'PeptideAbundance').rename_axis(columns=None).reset_index()
    pro_pep_df = pro.merge(pep, on = ['visit_id'], how = 'left')
    tt = pro_pep_df
    
    # test_proteins = convertUni(test_proteins)
    # test_df = test_proteins
    # test_df.fillna(0, inplace=True)
    # test_group = test_df.groupby(['visit_id','patient_id','visit_month','UniProt'])['NPX'].mean().reset_index()
    # tt = test_group.pivot(index='visit_id',columns='UniProt',values='NPX').rename_axis(columns=None).reset_index()
    
    test_df = test.merge(tt, how='left', on='visit_id')
    test_df.columns = test_df.columns.astype(str)
    test_df.fillna(0, inplace=True)
        
    return test_df

In [54]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    test_df = prepare(test, test_peptides, test_proteins)
    print(test_df)
    result = make_prediction(test_df, model)
    # result = make_time_prediction(test_df, lstm)
    # result = sort(result)
    env.predict(result)
    
print(result)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
  visit_id  visit_month  patient_id updrs_test           row_id   O00391  \
0   3342_0            0        3342    updrs_1   3342_0_updrs_1      0.0   
1   3342_0            0        3342    updrs_2   3342_0_updrs_2      0.0   
2   3342_0            0        3342    updrs_3   3342_0_updrs_3      0.0   
3   3342_0            0        3342    updrs_4   3342_0_updrs_4      0.0   
4  50423_0            0       50423    updrs_1  50423_0_updrs_1  33127.9   
5  50423_0            0       50423    updrs_2  50423_0_updrs_2  33127.9   
6  50423_0            0       50423    updrs_3  50423_0_updrs_3  33127.9   
7  50423_0            0       50423    updrs_4  50423_0_updrs_4  33127.9   

     O00533   O00584   O14773   O14791  ...  YSSDYFQAPSDYR  YTTEIIK  \
0       0.0      0.0      0.0     0.00  ...            0.0      0.0   
1       0.0      0.0      0.0     0.00  ...     

/tmp/ipykernel_25/30183174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = 0
/tmp/ipykernel_25/30183174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = 0
/tmp/ipykernel_25/30183174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  visit_id  visit_month  patient_id updrs_test           row_id   O00391  \
0   3342_6            6        3342    updrs_1   3342_6_updrs_1  14564.0   
1   3342_6            6        3342    updrs_2   3342_6_updrs_2  14564.0   
2   3342_6            6        3342    updrs_3   3342_6_updrs_3  14564.0   
3   3342_6            6        3342    updrs_4   3342_6_updrs_4  14564.0   
4  50423_6            6       50423    updrs_1  50423_6_updrs_1      0.0   
5  50423_6            6       50423    updrs_2  50423_6_updrs_2      0.0   
6  50423_6            6       50423    updrs_3  50423_6_updrs_3      0.0   
7  50423_6            6       50423    updrs_4  50423_6_updrs_4      0.0   

     O00533   O00584   O14498   O14773  ...  YSSDYFQAPSDYR  YTTEIIK  \
0  616804.0  61418.1  27830.1  14959.2  ...        31145.7  10359.8   
1  616804.0  61418.1  27830.1  14959.2  ...        31145.7  10359.8   
2  616804.0  61418.1  27830.1  14959.2  ...        31145.7  10359.8   
3  616804.0  61418.1  27830.1  

/tmp/ipykernel_25/30183174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = 0
/tmp/ipykernel_25/30183174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = 0
/tmp/ipykernel_25/30183174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                    prediction_id     rating
0    3342_6_updrs_1_plus_0_months   6.282953
4   50423_6_updrs_1_plus_0_months   6.718954
0    3342_6_updrs_2_plus_0_months  16.487715
4   50423_6_updrs_2_plus_0_months  25.150385
0    3342_6_updrs_3_plus_0_months   3.480000
4   50423_6_updrs_3_plus_0_months  13.170000
0    3342_6_updrs_4_plus_0_months   1.920000
4   50423_6_updrs_4_plus_0_months   5.130000
0    3342_6_updrs_1_plus_6_months   6.282953
4   50423_6_updrs_1_plus_6_months   6.718954
0    3342_6_updrs_2_plus_6_months  16.487715
4   50423_6_updrs_2_plus_6_months  25.150385
0    3342_6_updrs_3_plus_6_months   3.480000
4   50423_6_updrs_3_plus_6_months  13.170000
0    3342_6_updrs_4_plus_6_months   1.920000
4   50423_6_updrs_4_plus_6_months   5.130000
0   3342_6_updrs_1_plus_12_months   6.282953
4  50423_6_updrs_1_plus_12_months   6.718954
0   3342_6_updrs_2_plus_12_months  16.487715
4  50423_6_updrs_2_plus_12_months  25.150385
0   3342_6_updrs_3_plus_12_months   3.480000
4  50423_6

/tmp/ipykernel_25/30183174.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
/tmp/ipykernel_25/30183174.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["rating"] = temp["result_updrs_" + str(u)]
/tmp/ipykernel_25/30183174.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(temp)
/tmp/ipykern

In [55]:
result

,prediction_id,rating
0,3342_6_updrs_1_plus_0_months,6.282953
4,50423_6_updrs_1_plus_0_months,6.718954
0,3342_6_updrs_2_plus_0_months,16.487715
4,50423_6_updrs_2_plus_0_months,25.150385
0,3342_6_updrs_3_plus_0_months,3.480000
4,50423_6_updrs_3_plus_0_months,13.170000
0,3342_6_updrs_4_plus_0_months,1.920000
4,50423_6_updrs_4_plus_0_months,5.130000
0,3342_6_updrs_1_plus_6_months,6.282953
4,50423_6_updrs_1_plus_6_months,6.718954
